In [ ]:
pip install rdflib

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
import pandas as pd #read xslx
from rdflib import Graph, Literal, RDF, URIRef, Namespace 
from rdflib.namespace import FOAF , XSD, DC, FOAF, SKOS, RDF, RDFS
import urllib.parse 
from google.colab import files 
import io
import regex as re
import datetime


## Input file

In [ ]:
uploaded = files.upload()

In [ ]:
df = pd.read_excel('SP_DownloadLijst_LF_Contacten_WIP_20210108.xlsx', sheet_name='DATA')

In [ ]:
gp = pd.read_excel('gemeente-provincie.xlsx', sheet_name='Feuil2')

In [ ]:
df

In [ ]:
df.info()

In [ ]:
g = Graph()

## Helpers Fuctions

### mail_cleansing

In [ ]:
def mail_cleansing(mail):
  mail_cleansed = comment = np.nan

  if mail != 'nan':
    if  re.match(r'[\w\.-]+@[\w\.-]+(\.[\w]+)+', mail):
      mail_cleansed = mail
    else: 
      comment = 'Wrong mail format. Check it.'

  return [mail_cleansed, comment]

### referentieorganisatie_cleansing

In [ ]:
def referentieorganisatie_cleansing(referentieorganisatie):
  return re.sub(r'\s', '', referentieorganisatie)

### naam_contact_cleansing

In [ ]:
def naam_contact_cleansing(naam_contact):
  naam_contact_cleansed = comment = np.nan

  if naam_contact != 'nan':
    naam_contact = naam_contact.strip().title()

    if re.match(r'^[a-zA-ZàáâäãåąčćęèéêëėįìíîïłńòóôöõøùúûüųūÿýżźñçčšžÀÁÂÄÃÅĄĆČĖĘÈÉÊËÌÍÎÏĮŁŃÒÓÔÖÕØÙÚÛÜŲŪŸÝŻŹÑßÇŒÆČŠŽ∂ð \'’-]+$', naam_contact):
      naam_contact_cleansed = naam_contact
    else: 
      comment = 'Wrong naam format. Check it.'

  return [naam_contact_cleansed, comment]

### telephone_number_cleansing

In [ ]:
def telephone_number_cleansing(telephone_number):
  telephone_number = re.sub(r'\s', '', telephone_number)

  telephone_number = re.sub(r'^\+32-\(0\)', '0', telephone_number)
  telephone_number = re.sub(r'^\+32', '0', telephone_number)
  telephone_number = re.sub(r'^32', '0', telephone_number)

  telephone_number = re.sub(r'^\+0032-\(0\)', '0', telephone_number)
  telephone_number = re.sub(r'^0032-\(0\)', '0', telephone_number)
  telephone_number = re.sub(r'^0032', '0', telephone_number)

  telephone_number = re.sub(r'^\(\d\d\d\)', '0', telephone_number)
  telephone_number = re.sub(r'^0\(\d\d\d\)', '0', telephone_number)

  telephone_number =  re.sub(r'[\.a-zA-Z]', '', telephone_number)

  return split_telephone_number(telephone_number)

def split_telephone_number(telephone_number):
  telephone_number_1 = telephone_number_2 = np.nan
  comment = []

  split = [telephone_number]
  if '//' in telephone_number:
    split = telephone_number.split('//')

  split[0] = split[0].replace('/', '')

  if telephone_number != '' :
    if check_telephone_number_lenght(split[0]):
      telephone_number_1 = split[0]
    else:
      comment.append('Wrong telephone number lenght. Check it.')

    if len(split) == 2 :
      comment.append('Splitting. Check it.')
      split[1] = split[1].replace('/', '')
      if check_telephone_number_lenght(split[1]):
        telephone_number_2 = split[1]
      else:
        comment.append('Wrong telephone 2 number lenght. Check it.')

  return [telephone_number_1, telephone_number_2, ' - '.join(comment)]

def check_telephone_number_lenght(telephone_number):
  if len(telephone_number) < 9 or len(telephone_number) > 10:
    return False
  else :
    return True

### split_house_bus_number

In [ ]:
def split_house_bus_number(house_bus_number):
  house_number = bus_number = np.nan
  comment = []
  house_bus_number = house_bus_number.replace(' ', '')

  if ('z/n' not in house_bus_number and 'nan' not in house_bus_number) : 
    if (('bus' in house_bus_number or '/' in house_bus_number)) :
      comment.append('Splitting. Check it.')
      if ('bus' in house_bus_number) : 
        split = house_bus_number.split('bus')
      else :
        split = house_bus_number.split('/')
      house_number = split[0]
      bus_number = split[1]
    else:
        house_number = house_bus_number
    house_number =  house_number.replace('/', '').replace('-', '').replace(',', '')
      
  return [house_number, bus_number, ' - '.join(comment)]

### decretale_functie_cleasing


In [ ]:
def decretale_functie_cleasing(decretale):
  decretale_functie = functionaris_status = np.nan

  if 'Waarnemend' in decretale:
    status = decretale[decretale.find("("):decretale.find(")")+1]
    decretale_functie = decretale.replace(status, '').strip()
    functionaris_status = 'Waarnemend'
  elif decretale == 'GEEN of ONBEKEND':
    pass
  else:
    decretale_functie = decretale
    functionaris_status = 'Effectief'

  return [decretale_functie, functionaris_status]

### concept_uri

In [ ]:
import hashlib
import uuid

def concept_uri(base_uri, input):
  m = hashlib.md5()
  m.update(input.encode('utf-8'))

  return URIRef(base_uri + m.hexdigest())

def addLiteral(subject, predicate, object_value, datatype=None):
  if object_value != str(np.nan):
    if datatype == None:
      g.add((subject, predicate, Literal(object_value, lang='nl')))
    else:
      g.add((subject, predicate, Literal(object_value, datatype=datatype)))

In [ ]:
def existsContact(row):
  return ((str(row['Titel Cleansed']) != str(np.nan)) or (str(row['Mail nr2 Cleansed']) != str(np.nan)) or (str(row['Telefoonnr Contact 1']) != str(np.nan)))

## Data cleansing

### Titel

In [ ]:
df['Titel'].unique()

In [ ]:
df[['Titel Cleansed', 'Titel Comment']] = pd.DataFrame(df['Titel'].astype(str).apply(mail_cleansing).values.tolist(), columns=['titel_cleansed','comment'])

In [ ]:
df[df['Titel Comment'].str.contains('\w', na=False)]

### Referentieorganisatie

In [ ]:
df['Referentieorganisatie'].unique()

In [ ]:
df[df['Referentieorganisatie'].str.contains(r"\s", na=False)]

In [ ]:
df[df['Referentieorganisatie'].isnull()]

In [ ]:
df[['Referentieorganisatie Cleansed']] = pd.DataFrame(df['Referentieorganisatie'].astype(str).apply(referentieorganisatie_cleansing).values.tolist())

In [ ]:
df[df['Referentieorganisatie Cleansed'].str.contains(r"\s", na=False)]

In [ ]:
df['Referentieorganisatie Cleansed'].unique()

### Decretale functie

In [ ]:
df['Decretale functie'].unique()

In [ ]:
df[df['Decretale functie'].isnull()]

In [ ]:
#df['Decretale functie Cleansed'] = df[df['Decretale functie'] != 'Griffier']['Decretale functie'].str.title().replace('Geen Of Onbekend', np.NAN)

In [ ]:
df['Decretale functie Cleansed'].unique()

In [ ]:
df[['Decretale functie Cleansed', 'Functionaris status']] = pd.DataFrame(df[df['Decretale functie'] != 'Griffier']['Decretale functie'].astype(str).apply(decretale_functie_cleasing).values.tolist(), columns=['decretale cleansed','status'])

In [ ]:
df['Decretale functie Cleansed'].unique()

In [ ]:
df['Functionaris status'].unique()

### Familienaam Contact

In [ ]:
df['Familienaam Contact'].unique()

In [ ]:
df[~df['Familienaam Contact'].str.contains('^[a-zA-ZàáâäãåąčćęèéêëėįìíîïłńòóôöõøùúûüųūÿýżźñçčšžÀÁÂÄÃÅĄĆČĖĘÈÉÊËÌÍÎÏĮŁŃÒÓÔÖÕØÙÚÛÜŲŪŸÝŻŹÑßÇŒÆČŠŽ∂ð \'’-]+$', na=False)]

In [ ]:
df[['Familienaam Contact Cleansed', 'Familienaam Contact Comment']] = pd.DataFrame(df['Familienaam Contact'].astype(str).apply(naam_contact_cleansing).values.tolist(), columns=['naam_contact_cleansed','comment'])

In [ ]:
df[df['Familienaam Contact Comment'].str.contains("\w", na=False)]

In [ ]:
df['Familienaam Contact Cleansed'].unique()

### Voornaam Contact

In [ ]:
df['Voornaam Contact'].unique()

In [ ]:
df[~df['Voornaam Contact'].str.contains('^[a-zA-ZàáâäãåąčćęèéêëėįìíîïłńòóôöõøùúûüųūÿýżźñçčšžÀÁÂÄÃÅĄĆČĖĘÈÉÊËÌÍÎÏĮŁŃÒÓÔÖÕØÙÚÛÜŲŪŸÝŻŹÑßÇŒÆČŠŽ∂ð \'’-]+$', na=False)]

In [ ]:
df[['Voornaam Contact Cleansed', 'Voornaam Contact Comment']] = pd.DataFrame(df['Voornaam Contact'].astype(str).apply(naam_contact_cleansing).values.tolist(), columns=['naam_contact_cleansed','comment'])

In [ ]:
df[df['Voornaam Contact Comment'].str.contains("\w", na=False)]

In [ ]:
df['Voornaam Contact Cleansed'].unique()

### Telefoonnr Contact

**Not usual Format**

* 0032-(0)25538594
* +0032-(0)25527720
* 013/612174
* 016-267224
* 02/3590703	
* 052399893//052469213
* 056 73 70 19056/73 70 19 << duplicated
* +3258429736 058/42.97.36 << duplicated

In [ ]:
df[['Telefoonnr Contact 1', 'Telefoonnr Contact 2', 'Telefoonnr Contact Comment']] = pd.DataFrame(df['Telefoonnr Contact'].astype(str).apply(telephone_number_cleansing).values.tolist(), columns=['telephone_number_1', 'telephone_number_2', 'comment'])

In [ ]:
df[df['Telefoonnr Contact 1'].str.contains(r'\d', na=False)]

In [ ]:
df[df['Telefoonnr Contact 2'].str.contains(r'\d', na=False)]

In [ ]:
df[df['Telefoonnr Contact Comment'].str.contains('\w', na=False) & ~((df['Telefoonnr Contact'].str.contains('^0$', na=False)) | (df['Telefoonnr Contact'].isnull()))]

In [ ]:
df[(df['Telefoonnr Contact'].str.contains('^0$', na=False)) | (df['Telefoonnr Contact'].isnull())]

### GSMnr Contact

**Not usual Format**

* (486)079517
*  0488/981455 	

In [ ]:
df[['GSMnr Contact Cleansed', 'GSMnr Contact 2', 'GSMnr Contact Comment']] = pd.DataFrame(df['GSMnr Contact'].astype(str).apply(telephone_number_cleansing).values.tolist(), columns=['telephone_number_1', 'telephone_number_2', 'comment'])

In [ ]:
df[~(df['GSMnr Contact Cleansed'].str.len() == 10) & (df['GSMnr Contact Cleansed'].notnull())]

### Functietitel

In [ ]:
df['Functietitel Cleansed'] = df['Functietitel'].str.strip().str.title()
df['Functietitel Cleansed'] = df['Functietitel Cleansed'].replace('0', np.nan)

In [ ]:
df['Functietitel Cleansed'].unique()

In [ ]:
df[(df['Functietitel'].str.contains('^0$', na=False)) | (df['Functietitel'].isnull())]

### Secretaris

In [ ]:
df['Secretaris'].unique()

In [ ]:
df[df['Secretaris'].isnull()]

### Griffier

In [ ]:
df['Griffier'].unique()

In [ ]:
df[df['Griffier'].isnull()]

### Financieel Beheerder

In [ ]:
df['Financieel Beheerder'].unique()

In [ ]:
df[df['Financieel Beheerder'].isnull()]

### BBC Contact

In [ ]:
df['BBC Contact'].unique()

In [ ]:
df[df['BBC Contact'].isnull()]

### Banken Contact

In [ ]:
df['Banken Contact'].unique()

In [ ]:
df[df['Banken Contact'].isnull()]

### ICT Contact  

In [ ]:
df['ICT Contact'].unique()

In [ ]:
df[df['ICT Contact'].isnull()]

### Federaties Contact

In [ ]:
df['Federaties Contact'].unique()

In [ ]:
df[df['Federaties Contact'].isnull()]

### Werkt in:KBOnr

In [ ]:
df['Werkt in:KBOnr'].unique()

In [ ]:
df['Werkt in:KBOnr Cleansed'] = df['Werkt in:KBOnr'].replace('Onbekend', np.nan)

In [ ]:
df['organisation_id'] = df['Werkt in:KBOnr Cleansed'].fillna(df['Referentieorganisatie'])

In [ ]:
df[~(df['Werkt in:KBOnr'].str.len() == 10) & (df['Werkt in:KBOnr'].notnull())] #10 digit

In [ ]:
df[df['Werkt in:KBOnr'].isnull()]

### Werkt in:Straat

In [ ]:
df['Werkt in:Straat'].unique()

In [ ]:
df[df['Werkt in:Straat'].isnull()]

### Werkt in:Huisnr

**Huis Nummer & Bus Nummer**

* 29 bus 1
* 5 - bus2
* 141/2
* 1 / bus 1
* 4 -  bus2
* 51/2

**Huis Nummer**

* 6-7
* 4A


**Special case**

* 1/A000 >> bus nummer? | looks like it
* 13, 4de verdieping >> ?? | 4th floor
* '50, Cargovil', >> ?? | an industrial area? would ignore it
* '24, gebouw A' >> ?? | building A
* z/n

In [ ]:
df['Werkt in:Huisnr'].unique()

In [ ]:
df[df['Werkt in:Huisnr'].isnull()]

In [ ]:
df[['Huisnr', 'Busnr', 'Werkt in:Huisnr Comment']] = pd.DataFrame(df['Werkt in:Huisnr'].astype(str).apply(split_house_bus_number).values.tolist(), columns=['house_number', 'bus_number', 'comment'])
df[df['Werkt in:Huisnr Comment'].str.contains('\w', na=False)]

In [ ]:
df['Huisnr'].unique()

In [ ]:
df['Busnr'].unique()

### Werkt in:Postcode van de organisatie

In [ ]:
df['Werkt in:Postcode van de organisatie'] = df['Werkt in:Postcode van de organisatie'].astype(str).str.replace('\.0', '')
df['Werkt in:Postcode van de organisatie'] = df['Werkt in:Postcode van de organisatie'].replace('nan', np.NaN)
df['Werkt in:Postcode van de organisatie'].unique()

In [ ]:
df[~(df['Werkt in:Postcode van de organisatie'].str.len() == 4) & (df['Werkt in:Postcode van de organisatie'].notnull())] #4 digit

In [ ]:
df[df['Werkt in:Postcode van de organisatie'].isnull()]

### Werkt in:Gemeente van de organisatie

In [ ]:
df['Werkt in:Gemeente van de organisatie'] = df['Werkt in:Gemeente van de organisatie'].str.title()
df['Werkt in:Gemeente van de organisatie'].unique()

### Werkt in:Unieke Naam

In [ ]:
df[df['Werkt in:Unieke Naam'].str.strip().str.contains("/s", na=False)]

In [ ]:
df[df['Werkt in:Unieke Naam'].isnull()]

### Werkt in:TypeEntiteitCalc

In [ ]:
df['Werkt in:TypeEntiteitCalc'].unique()

In [ ]:
df[df['Werkt in:TypeEntiteitCalc'].isnull()]

### Werkt in:Provincie van de organisatie calc

In [ ]:
df['Werkt in:Provincie van de organisatie calc'].unique()

In [ ]:
df[(df['Werkt in:Provincie van de organisatie calc'].str.contains('^0$', na=False)) | (df['Werkt in:Provincie van de organisatie calc'].isnull())]

### Mail nr2

In [ ]:
df['Mail nr2'].unique()

In [ ]:
df[['Mail nr2 Cleansed', 'Mail nr2 Comment']] = pd.DataFrame(df['Mail nr2'].astype(str).apply(mail_cleansing).values.tolist(), columns=['mail_cleansed','comment'])

In [ ]:
df[df['Mail nr2 Comment'].str.contains('\w', na=False)]

In [ ]:
df[df['Mail nr2'].isnull()]

### Id

In [ ]:
df['Id Cleansed'] = df['Id']

In [ ]:
df[df['Id Cleansed'].isnull()]

### Contact Waarborgen

In [ ]:
df['Contact Waarborgen'].unique()

In [ ]:
df[df['Contact Waarborgen'].isnull()]

### Contact PPS

In [ ]:
df['Contact PPS'].unique()

In [ ]:
df[df['Contact PPS'].isnull()]

### Contact Deelnemingen

In [ ]:
df['Contact Deelnemingen'].unique()

In [ ]:
df[df['Contact Deelnemingen'].isnull()]

### Contact Mandaten

In [ ]:
df['Contact Mandaten'].unique()

In [ ]:
df[df['Contact Mandaten'].isnull()]

### Contact BBC-DR

In [ ]:
df['Contact BBC-DR'].unique()

In [ ]:
df[df['Contact BBC-DR'].isnull()]

### Status Contact Actief

In [ ]:
df['Status Contact Actief'].unique()

In [ ]:
df[df['Status Contact Actief'].isnull()]

### Referentieorganisatie:Organisatiestatus calc

In [ ]:
df['Referentieorganisatie:Organisatiestatus calc'].unique()

In [ ]:
df[df['Referentieorganisatie:Organisatiestatus calc'].isnull()]

### Itemtype

In [ ]:
df['Itemtype'].unique()

### Pad

In [ ]:
df['Pad'].unique()

# Export data

In [ ]:
df.to_excel('contacten.xlsx')

## Namespace

In [ ]:
org = Namespace('http://www.w3.org/ns/org#')
locn = Namespace('http://www.w3.org/ns/locn#')
dc_terms= Namespace('http://purl.org/dc/terms/')
schema = Namespace('http://schema.org/')
regorg = Namespace('http://www.w3.org/ns/regorg#')
person = Namespace('http://www.w3.org/ns/person#')
vcard = Namespace('http://www.w3.org/2006/vcard/ns#')
dbpedia = Namespace('https://dbpedia.org/ontology/')

organisatie = Namespace('https://data.vlaanderen.be/ns/organisatie')
persoon = Namespace('https://data.vlaanderen.be/ns/persoon#')
adres = Namespace('https://data.vlaanderen.be/ns/adres#')
generiek = Namespace('https://data.vlaanderen.be/ns/generiek#')
mandaat = Namespace('http://data.vlaanderen.be/ns/mandaat#')
besluit = Namespace('http://data.vlaanderen.be/ns/besluit#')
lblodlg = Namespace('http://data.lblod.info/vocabularies/leidinggevenden/')

#lblod = Namespace('https://contacthub-dev.lblod.info/id/')
lblod = Namespace('https://contacthub-qa.lblod.info/id/')
os = Namespace('https://data.vlaanderen.be/id/concept/organisatiestatus/')
oc = Namespace('https://data.vlaanderen.be/id/concept/OrganisatieClassificatie/')

functionaris_status = {'Waarnemend': URIRef('https://data.vlaanderen.be/id/conceptscheme/MandatarisStatusCode/e1ca6edd-55e1-4288-92a5-53f4cf71946a'), 
                        'Effectief': URIRef('https://data.vlaanderen.be/id/concept/MandatarisStatusCode/21063a5b-912c-4241-841c-cc7fb3c73e75')}
rol_concept = {'Algemeen Directeur': URIRef('https://data.vlaanderen.be/id/concept/BestuursfunctieCode/39e08271-68db-4282-897f-5cba88c71862'),
               'Financieel Directeur': URIRef('https://data.vlaanderen.be/id/concept/BestuursfunctieCode/6d4cf4dd-2080-4752-8733-d02a036b2df0'),
               'Adjunct Financieel Directeur': URIRef('https://data.vlaanderen.be/id/concept/BestuursfunctieCode/3200ffc1-bb72-4235-a81c-64aa578b0789'),
               'Adjunct Algemeen Directeur': URIRef('https://data.vlaanderen.be/id/concept/BestuursfunctieCode/f7b4e17b-6f4e-48e7-a558-bce61669f59a')}

# 


In [ ]:
g = Graph()

In [ ]:
df.info()

In [ ]:
df_cleansed = df[df['Familienaam Contact Cleansed'].notnull()]

## Mapping

**To do:**
 
Titel - OK

Referentieorganisatie - OK

Decretale functie - OK

Familienaam Contact - OK
 
Voornaam Contact - OK

Telefoonnr Contact - OK

GSMnr Contact Mobile phone? Don't have place for it. But put in new contact point

Functietitel - OK

Mail nr2 - OK

Id - OK

In [ ]:
for index, row in df_cleansed.iterrows():
  abb_id = concept_uri(lblod + 'persoon/', str(row['Voornaam Contact Cleansed']) + str(row['Familienaam Contact Cleansed']))

  g.add((abb_id, RDF.type, person.Person))
  addLiteral(abb_id, FOAF.familyName, str(row['Familienaam Contact Cleansed']), XSD.string)
  addLiteral(abb_id, persoon.gebruikteVoornaam, str(row['Voornaam Contact Cleansed']), XSD.string)

  if existsContact(row):
    site_id = concept_uri(lblod + 'vesting/', str(row['Voornaam Contact Cleansed']) + str(row['Familienaam Contact Cleansed']))
    g.add((site_id, RDF.type, org.Site))

    contact_id = concept_uri(lblod + 'contactinfo/', str(row['Voornaam Contact Cleansed']) + str(row['Familienaam Contact Cleansed']) + '1')
    g.add((contact_id, RDF.type, schema.ContactPoint))
    addLiteral(contact_id, schema.email, str(row['Titel Cleansed']), XSD.string)
    addLiteral(contact_id, schema.email, str(row['Mail nr2 Cleansed']), XSD.string)
    addLiteral(contact_id, schema.telephone, str(row['Telefoonnr Contact 1']), XSD.string) 
    g.add((abb_id, schema.contactPoint, contact_id))

    if str(row['Telefoonnr Contact 2']) != str(np.nan):
      contact_id = concept_uri(lblod + 'contactinfo/', str(row['Voornaam Contact Cleansed']) + str(row['Familienaam Contact Cleansed']) + 'Telefoonnr Contact 2')
      g.add((contact_id, RDF.type, schema.ContactPoint))
      addLiteral(contact_id, schema.telephone, str(row['Telefoonnr Contact 2']), XSD.string) 
      g.add((abb_id, schema.contactPoint, contact_id))

    if str(row['GSMnr Contact Cleansed']) != str(np.nan):
      contact_id = concept_uri(lblod + 'contactinfo/', str(row['Voornaam Contact Cleansed']) + str(row['Familienaam Contact Cleansed']) + 'GSMnr Contact Cleansed')
      g.add((contact_id, RDF.type, schema.ContactPoint))
      addLiteral(contact_id, schema.telephone, str(row['GSMnr Contact Cleansed']), XSD.string) 
      g.add((abb_id, schema.contactPoint, contact_id))
    
  if str(row['Id']) != str(np.nan):
    id = concept_uri(lblod + 'gestructureerdeIdentificator/', str(row['Voornaam Contact Cleansed']) + str(row['Familienaam Contact Cleansed']))
    g.add((id, RDF.type, generiek.GestructureerdeIdentificator))
    addLiteral(id, generiek.lokaleIdentificator, str(row['Id']), XSD.string)

    g.add((abb_id, generiek.gestructureerdeIdentificator, id))

  org_id = concept_uri(lblod + 'organisatie/', str(row['Werkt in:KBOnr Cleansed']))
  if (str(row['Decretale functie Cleansed']) == str(np.nan)) and (str(row['Functietitel Cleansed']) != str(np.nan)):
    position_id = concept_uri(lblod + 'hoedanigheid/', str(row['Functietitel Cleansed']))
    g.add((position_id, RDF.type, organisatie.Hoedanigheid))

    role_id = concept_uri(lblod + 'rol/', str(row['Functietitel Cleansed']))
    g.add((role_id, RDF.type, org.Role))
    addLiteral(role_id, RDFS.label, str(row['Functietitel Cleansed']))

    g.add((position_id, org.role, role_id))

    g.add((position_id, org.postIn, org_id))
    g.add((org_id, org.hasPost, position_id))

    g.add((abb_id, org.holds, position_id))
    g.add((position_id, org.heldBy, abb_id))
  elif str(row['Decretale functie Cleansed']) != str(np.nan):
    # Bestuur temporary
    bestuur_temporary = concept_uri(lblod + 'bestuursorgaan/', str(row['Werkt in:KBOnr Cleansed']) + str(datetime.datetime.now()))
    g.add((bestuur_temporary, RDF.type, besluit.Bestuursorgaan))
    g.add((bestuur_temporary, generiek.isTijdspecialisatieVan, org_id))

    ## Functionaris
    person_functionaris = concept_uri(lblod + 'functionaris/',  str(row['Voornaam Contact Cleansed']) + str(row['Familienaam Contact Cleansed']) + str(row['Werkt in:KBOnr Cleansed']) + str(row['Decretale functie Cleansed'].lower().replace(" ", "")))
    g.add((person_functionaris, RDF.type, lblodlg.Functionaris))
    g.add((abb_id, mandaat.isAangesteldAls, person_functionaris))
    g.add((person_functionaris, mandaat.isBestuurlijkeAliasVan, abb_id))
    #start
    #einde
    #status ~ cf loket lokale besturen PoC https://poc-form-builder.relance.s.redpencil.io/codelijsten
    # https://data.vlaanderen.be/id/conceptscheme/MandatarisStatusCode
    g.add((person_functionaris, mandaat.status, functionaris_status[row['Functionaris status']]))

    # https://data.vlaanderen.be/doc/conceptscheme/BestuursfunctieCode
    person_bestuursfunctie = concept_uri(lblod + 'bestuursfunctie/', str(row['Voornaam Contact Cleansed']) + str(row['Familienaam Contact Cleansed']) + str(row['Werkt in:KBOnr Cleansed']))
    g.add((person_bestuursfunctie, RDF.type, lblodlg.Bestuursfunctie))
    
    g.add((person_bestuursfunctie, org.role, rol_concept[row['Decretale functie Cleansed']]))
    g.add((person_functionaris, org.holds, person_bestuursfunctie))

    g.add((bestuur_temporary, org.hasPost, person_bestuursfunctie))

In [ ]:
g.serialize('contact-qa.ttl',format='turtle')